In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/datasets-for-federated-learning/wefe-default_data/customer/customers_subset_2.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/customer/customers_without_y.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/customer/customers_with_y.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/customer/customers_subset_1.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/customer/marketing_campaign.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/heart/heart_subset_2.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/heart/heart_without_y.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/heart/heart_with_y.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/heart/heart.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/heart/heart_subset_1.csv
/kaggle/input/datasets-for-federated-learning/wefe-default_data/machine/machine_with_y

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas
import sklearn
import xgboost
import sklearn.linear_model
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from datetime import datetime
from scipy.io.arff import loadarff

1. Body signal of smoking (XGB Model)

https://www.kaggle.com/datasets/kukuroo3/body-signal-of-smoking

license: CC0: Public Domain

In [4]:
smoking = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/smoking/smoking.csv")
smoking = smoking.drop(columns = "oral")
smoking

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar,smoking
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,0,Y,0
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,0,Y,0
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,151.0,15.8,1.0,1.0,21.0,16.0,22.0,0,N,1
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,226.0,14.7,1.0,1.0,19.0,26.0,18.0,0,Y,0
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,107.0,12.5,1.0,0.6,16.0,14.0,22.0,0,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55687,55676,F,40,170,65,75.0,0.9,0.9,1.0,1.0,...,118.0,12.3,1.0,0.6,14.0,7.0,10.0,1,Y,0
55688,55681,F,45,160,50,70.0,1.2,1.2,1.0,1.0,...,79.0,14.0,1.0,0.9,20.0,12.0,14.0,0,Y,0
55689,55683,F,55,160,50,68.5,1.0,1.2,1.0,1.0,...,63.0,12.4,1.0,0.5,17.0,11.0,12.0,0,N,0
55690,55684,M,60,165,60,78.0,0.8,1.0,1.0,1.0,...,146.0,14.4,1.0,0.7,20.0,19.0,18.0,0,N,0


In [5]:
#hard coding
smoking["gender"] = smoking["gender"].map({"M":1, "F":0})
smoking["tartar"] = smoking["tartar"].map({"Y":1, "N":0})

#standardization
x = smoking[smoking.columns[1:-1]]
y = smoking["smoking"]
ss = StandardScaler()
x = ss.fit_transform(x)

#data split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#XGB
xgb_cls = xgboost.XGBClassifier()
xgb_clf = xgb_cls.fit(train_x, train_y)

#prediction, print evaluation results
xgb_predict_y = xgb_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(xgb_predict_y, test_y)
print("\nXGB accuracy: {}\nXGB auc: {}\nKS: {}".format(sum(xgb_predict_y == test_y)/len(xgb_predict_y), 
                                             sklearn.metrics.roc_auc_score(xgb_predict_y, test_y),
                                                    max(tpr-fpr)))


XGB accuracy: 0.7830979171654298
XGB auc: 0.7662426326574294
KS: 0.5324852653148588


2. Heart Attack Analysis & Prediction Dataset (LR Model)

https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset

license: CC0: Public Domain

In [6]:
heart = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/heart/heart.csv")
heart

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [7]:
#woe coding
def woe(data):
    import math
    large1, large0 = sum(data[data.columns[-1]]), sum(data[data.columns[-1]] == False)
    for column in data.columns[:-1]:
        local = {}
        for unique in set(data[column].unique()):
            local_1, local_0 = sum(data[data[column] == unique][data.columns[-1]]), sum(data[data[column] == unique][data.columns[-1]] == False)
            if local_1 == 0:
                local[unique] = 0
            elif local_0 != 0:
                local[unique] = (
                                    math.log(
                                            (local_1/large1)/(local_0/large0)  
                                            )
                                )
            else:
                local[unique] = (
                                (local_1/large1)
                                )
        temp = []
        for row in data[column]:
            temp.append(local[row])
        data[column] = temp
    return data

num = [i for i in heart.columns if len(heart[i].unique()) > 5]

processed = woe(heart[[i for i in heart if i not in num]])
heart[[i for i in heart.columns if i not in num]] = processed

#starnardization
heart[num] = heart[num].apply(lambda x:(x-x.mean())/x.std())

#data split
x = heart[heart.columns[:-1]]
y = heart[heart.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#LR
lr_cls = sklearn.linear_model.LogisticRegression()
lr_clf = lr_cls.fit(train_x, train_y)

#prediction, print evaluation results
lr_predict_y = lr_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(lr_predict_y, test_y)
print("\nLR accuracy: {}\nLR auc: {}\nKS: {}".format(sum(lr_predict_y == test_y)/len(lr_predict_y), 
                                             sklearn.metrics.roc_auc_score(lr_predict_y, test_y),
                                                    max(tpr-fpr)))


LR accuracy: 0.8681318681318682
LR auc: 0.8717171717171717
KS: 0.7434343434343433


3. Water Quality (LR Model)

https://www.kaggle.com/datasets/adityakadiwal/water-potability

license: CC0: Public Domain

In [8]:
water = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/water/water_potability.csv")
water

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


In [9]:
#filling missing value
for i in water.columns:
    if sum(water[i].isnull()) > 0:
        water[i] = water[i].fillna(water[i].mean())

#standardization
x = water[water.columns[:-1]]
y = water[water.columns[-1]]
x = x.apply(lambda x:(x - x.mean())/x.std())

#data split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#LR
lr_cls = sklearn.linear_model.LogisticRegression()
lr_clf = lr_cls.fit(train_x, train_y)

#prediction, print evaluation results
lr_predict_y = lr_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(lr_predict_y, test_y)
print("\nLR accuracy: {}\nLR auc: {}\nKS: {}".format(sum(lr_predict_y == test_y)/len(lr_predict_y), 
                                             sklearn.metrics.roc_auc_score(lr_predict_y, test_y),
                                                    max(tpr-fpr)))


LR accuracy: 0.6113936927772126
LR auc: 0.8054989816700611
KS: 0.6109979633401221


4. Customer Personality Analysis (LR Model)

https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis

license: CC0: Public Domain

In [10]:
customer = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/customer/marketing_campaign.csv", sep="\t")
customer = customer.drop(columns = ["Z_CostContact", "Z_Revenue", "Dt_Customer"])
customer

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,58,635,88,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,38,11,1,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,26,426,49,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,26,11,4,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,PhD,Married,58293.0,1,0,94,173,43,...,3,6,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,46,709,43,...,3,4,5,0,0,0,0,0,0,0
2236,4001,1946,PhD,Together,64014.0,2,1,56,406,0,...,2,5,7,0,0,0,1,0,0,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,91,908,48,...,3,13,6,0,1,0,0,0,0,0
2238,8235,1956,Master,Together,69245.0,0,1,8,428,30,...,5,10,3,0,0,0,0,0,0,0


In [11]:
#woe coding
processed = woe(customer[["Education", "Marital_Status", "Response"]])
customer[["Education", "Marital_Status", "Response"]] = processed

#filling missing value
customer["Income"] = customer["Income"].fillna(customer["Income"].mean())

#standardization
numf = [i for i in customer.columns if i not in ["ID", "Education", "Marital_Status", "Response"]]
standardize = customer[numf]
ss = StandardScaler()
standardize = ss.fit_transform(standardize)
customer[numf] = standardize

#data split
x = customer[customer.columns[1:-1]]
y = customer[customer.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#LR
lr_cls = sklearn.linear_model.LogisticRegression()
lr_clf = lr_cls.fit(train_x, train_y)

#prediction, print evaluation results
lr_predict_y = lr_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(lr_predict_y, test_y)
print("\nLR accuracy: {}\nLR auc: {}\nKS: {}".format(sum(lr_predict_y == test_y)/len(lr_predict_y), 
                                             sklearn.metrics.roc_auc_score(lr_predict_y, test_y),
                                                    max(tpr-fpr)))


LR accuracy: 0.8779761904761905
LR auc: 0.8017369727047148
KS: 0.6034739454094293


5. Travel Insurance Prediction Data (XGB Model)

https://www.kaggle.com/datasets/tejashvi14/travel-insurance-prediction-data

license: CC0: Public Domain

In [12]:
insurance = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/insurance/TravelInsurancePrediction.csv")
insurance

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,0,31,Government Sector,Yes,400000,6,1,No,No,0
1,1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1
3,3,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
4,4,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0
...,...,...,...,...,...,...,...,...,...,...
1982,1982,33,Private Sector/Self Employed,Yes,1500000,4,0,Yes,Yes,1
1983,1983,28,Private Sector/Self Employed,Yes,1750000,5,1,No,Yes,0
1984,1984,28,Private Sector/Self Employed,Yes,1150000,6,1,No,No,0
1985,1985,34,Private Sector/Self Employed,Yes,1000000,6,0,Yes,Yes,1


In [13]:
#binning
num = ["Age", "FamilyMembers"]
bins = KBinsDiscretizer(n_bins = 4, strategy = "uniform", encode = "ordinal")
bins = bins.fit_transform(insurance[num])
insurance[num] = bins

#woe coding
to_woe = [i for i in insurance.columns if i not in ["Unnamed: 0","AnnualIncome"]]
processed = woe(insurance[to_woe])
insurance[to_woe] = processed

#standardization
insurance["AnnualIncome"] = insurance["AnnualIncome"].apply(lambda x: (x - insurance["AnnualIncome"].mean())/insurance["AnnualIncome"].std())

#data split
x = insurance[insurance.columns[1:-1]]
y = insurance[insurance.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#XGB
xgb_cls = xgboost.XGBClassifier()
xgb_clf = xgb_cls.fit(train_x, train_y)

#prediction, print evaluation
xgb_predict_y = xgb_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(xgb_predict_y, test_y)
print("\nXGB accuracy: {}\nXGB auc: {}\nKS: {}".format(sum(xgb_predict_y == test_y)/len(xgb_predict_y), 
                                               sklearn.metrics.roc_auc_score(xgb_predict_y, test_y),
                                              max(tpr-fpr)))


XGB accuracy: 0.8040201005025126
XGB auc: 0.8223873441994247
KS: 0.6447746883988494


6. The Home Equity dataset (XGB Model)

https://www.kaggle.com/datasets/ajay1735/hmeq-data

license: CC0: Public Domain

In [14]:
credit = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/credit/hmeq.csv")
credit

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,0,88900,57264.0,90185.0,DebtCon,Other,16.0,0.0,0.0,221.808718,0.0,16.0,36.112347
5956,0,89000,54576.0,92937.0,DebtCon,Other,16.0,0.0,0.0,208.692070,0.0,15.0,35.859971
5957,0,89200,54045.0,92924.0,DebtCon,Other,15.0,0.0,0.0,212.279697,0.0,15.0,35.556590
5958,0,89800,50370.0,91861.0,DebtCon,Other,14.0,0.0,0.0,213.892709,0.0,16.0,34.340882


In [15]:
#filling missing value
credit["REASON"] = credit["REASON"].fillna(credit["REASON"].mode()[0])
credit["JOB"] = credit["JOB"].fillna(credit["JOB"].mode()[0])
for i in ["MORTDUE", "VALUE", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO"]:
    credit[i] = credit[i].fillna(credit[i].mean())
credit["DEBTINC"] = credit["DEBTINC"].apply(lambda x:x if x > 0 else -1)

#binning
num = ["LOAN", "MORTDUE", "VALUE", "YOJ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
bins = KBinsDiscretizer(n_bins = 4, strategy = "quantile", encode = "ordinal")
bins = bins.fit_transform(credit[num])
credit[num] = bins

num = ["DEROG"]
bins = KBinsDiscretizer(n_bins = 4, strategy = "uniform", encode = "ordinal")
bins = bins.fit_transform(credit[num])
credit[num] = bins

#woe coding
to_woe = [i for i in credit.columns[1:] if i != "DELINQ"] + ["BAD"]
processed = woe(credit[to_woe])
credit[to_woe] = processed

#standardization
credit["DELINQ"] = credit["DELINQ"].apply(lambda x: (x - credit["DELINQ"].mean())/credit["DELINQ"].std())

#data split
x = credit[credit.columns[1:]]
y = credit[credit.columns[0]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#XGB
xgb_cls = xgboost.XGBClassifier()
xgb_clf = xgb_cls.fit(train_x, train_y)

#prediction, print evaluation results
xgb_predict_y = xgb_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(xgb_predict_y, test_y)
print("\nXGB accuracy: {}\nXGB auc: {}\nKS: {}".format(sum(xgb_predict_y == test_y)/len(xgb_predict_y), 
                                               sklearn.metrics.roc_auc_score(xgb_predict_y, test_y),
                                              max(tpr-fpr)))


XGB accuracy: 0.8965324384787472
XGB auc: 0.859309684661419
KS: 0.7186193693228381


7. Income Dataset (XGB Model)

https://www.kaggle.com/datasets/mastmustu/income

license: CC0: Public Domain

In [16]:
income = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/income/train.csv")
income

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States,1
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,0
2,31,Private,174201,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
3,58,State-gov,110199,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,0
4,25,State-gov,149248,Some-college,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43952,52,Private,68982,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,1
43953,19,Private,116562,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,0
43954,30,Private,197947,Some-college,10,Divorced,Sales,Not-in-family,White,Male,0,0,58,United-States,0
43955,46,Private,97883,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,0,35,United-States,0


In [17]:
#filling missing value
income["workclass"] = income["workclass"].fillna("-1")
income["occupation"] = income["occupation"].fillna("-1")

#hard coding
developed = ["United-States", "Japan", "South", "Portugal", 
 "Italy", "England", "Germany", "Yugoslavia", 
 "Poland", "Greece", "Ireland", "Canada", 
 "Scotland", "Outlying-US(Guam-USVI-etc)", "Taiwan", "France",
 "Hungary", "Hong", "Holand-Netherlands"]
income["native-country"] = income["native-country"].apply(lambda x: 1 if x in developed else 0)

#binning
num = ["age", "hours-per-week"]
bins = KBinsDiscretizer(n_bins = 4, strategy = "uniform", encode = "ordinal")
bins = bins.fit_transform(income[num])
income[num] = bins

num = ["educational-num"]
bins = KBinsDiscretizer(n_bins = 5, strategy = "quantile", encode = "ordinal")
bins = bins.fit_transform(income[num])
income[num] = bins

#woe coding
numf = ["fnlwgt", "capital-gain", "capital-loss"]

processed = woe(income[[i for i in income.columns if i not in numf]])
income[[i for i in income.columns if i not in numf]] = processed

#standardization
income[numf] = income[numf].apply(lambda x: (x-x.mean())/x.std())

#data split
x = income[income.columns[:-1]]
y = income[income.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#XGB
xgb_cls = xgboost.XGBClassifier()
xgb_clf = xgb_cls.fit(train_x, train_y)

#prediction, print evaluation results
xgb_predict_y = xgb_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(xgb_predict_y, test_y)
print("\nXGB accuracy: {}\nXGB auc: {}\nKS: {}".format(sum(xgb_predict_y == test_y)/len(xgb_predict_y), 
                                               sklearn.metrics.roc_auc_score(xgb_predict_y, test_y),
                                              max(tpr-fpr)))


XGB accuracy: 0.8681377009402487
XGB auc: 0.8350725340614357
KS: 0.6701450681228714


8. Machine Predictive Maintenance Classification (LR Model)

https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification

license: CC0: Public Domain

In [18]:
machine = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/machine/predictive_maintenance.csv")
machine["y"] = machine["Target"]
machine = machine.drop(columns = ["Target", "Failure Type", "Product ID"])
machine

,UDI,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,y
0,1,M,298.1,308.6,1551,42.8,0,0
1,2,L,298.2,308.7,1408,46.3,3,0
2,3,L,298.1,308.5,1498,49.4,5,0
3,4,L,298.2,308.6,1433,39.5,7,0
4,5,L,298.2,308.7,1408,40.0,9,0
...,...,...,...,...,...,...,...,...
9995,9996,M,298.8,308.4,1604,29.5,14,0
9996,9997,H,298.9,308.4,1632,31.8,17,0
9997,9998,M,299.0,308.6,1645,33.4,22,0
9998,9999,H,299.0,308.7,1408,48.5,25,0


In [19]:
#binning
num = ["Air temperature","Process temperature", "Rotational speed", "Torque", "Tool wear"]
bins = KBinsDiscretizer(n_bins = 4, strategy = "uniform", encode = "ordinal")
bins = bins.fit_transform(machine[num])
machine[num] = bins

#woe coding
processed = woe(machine[[i for i in machine.columns if i != "UDI"]])
machine[[i for i in machine.columns if i != "UDI"]] = processed

#data split
x = machine[machine.columns[1:-1]]
y = machine[machine.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#LR
lr_cls = sklearn.linear_model.LogisticRegression()
lr_clf = lr_cls.fit(train_x, train_y)

#prediction, print evaluation results
lr_predict_y = lr_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(lr_predict_y, test_y)
print("\nLR accuracy: {}\nLR auc: {}\nKS: {}".format(sum(lr_predict_y == test_y)/len(lr_predict_y), 
                                             sklearn.metrics.roc_auc_score(lr_predict_y, test_y),
                                                    max(tpr-fpr)))


LR accuracy: 0.9716666666666667
LR auc: 0.8739610382555222
KS: 0.7479220765110443


9. Lumpy Skin Disease Dataset (LR Model)

license: Attribution 4.0 International (CC BY 4.0)

https://www.kaggle.com/datasets/saurabhshahane/lumpy-skin-disease-dataset

In [20]:
skin = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/skin/Lumpy skin disease data.csv")
skin["param1"] = skin["x"]
skin["param2"] = skin["y"]
skin = skin.drop(columns = ["x", "y", "region", "country", "reportingDate"])
skin["y"] = skin["lumpy"]
skin = skin.drop(columns = "lumpy")
skin

,cld,dtr,frs,pet,pre,tmn,tmp,tmx,vap,wet,elevation,dominant_land_cover,X5_Ct_2010_Da,X5_Bf_2010_Da,param1,param2,y
0,41.6,12.8,0.00,2.3,1.7,12.7,19.1,25.5,15.7,0.00,147,2,27970.983100,3691.746950,90.380931,22.437184,1
1,40.5,13.3,0.00,2.4,0.0,13.2,19.8,26.5,16.3,0.00,145,2,25063.646690,671.326701,87.854975,22.986757,1
2,27.3,13.6,0.08,2.3,0.6,9.4,16.2,23.0,13.0,0.98,158,2,6038.477155,1426.839831,85.279935,23.610181,1
3,45.3,12.8,31.00,0.4,8.8,-22.5,-16.1,-9.7,0.9,4.64,178,2,760.703340,0.000000,81.564510,43.882221,1
4,38.8,13.2,31.00,0.4,10.5,-20.4,-13.8,-7.2,1.2,1.69,185,3,270.367426,0.000000,81.161057,43.834976,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24798,68.8,12.0,0.00,3.6,172.0,17.7,23.7,29.7,22.4,15.60,184,3,95.184093,0.000000,20.250000,-16.750000,0
24799,68.8,11.8,0.00,3.6,170.0,18.0,23.9,29.8,23.2,15.70,182,4,111.561489,0.000000,20.750000,-16.750000,0
24800,68.8,11.6,0.00,3.6,167.5,18.3,24.1,29.9,23.9,15.87,178,5,109.733287,0.000000,21.250000,-16.750000,0
24801,68.8,11.5,0.00,3.6,169.5,18.5,24.2,30.0,24.2,16.30,180,3,103.753469,0.000000,21.750000,-16.750000,0


In [21]:
#woe coding
processed = woe(skin[["dominant_land_cover", "y"]])
skin[["dominant_land_cover", "y"]] = processed

#standardization
skin[[i for i in skin.columns[:-1] if i != "dominant_land_cover"]] = skin[[i for i in skin.columns[:-1] if i != "dominant_land_cover"]].apply(lambda x:(x-x.mean())/x.std())

#data split
x = skin[skin.columns[:-1]]
y = skin[skin.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#LR
lr_cls = sklearn.linear_model.LogisticRegression()
lr_clf = lr_cls.fit(train_x, train_y)

#prediction, print evaluation results
lr_predict_y = lr_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(lr_predict_y, test_y)
print("\nLR accuracy: {}\nLR auc: {}\nKS: {}".format(sum(lr_predict_y == test_y)/len(lr_predict_y), 
                                             sklearn.metrics.roc_auc_score(lr_predict_y, test_y),
                                                    max(tpr-fpr)))


LR accuracy: 0.9469157371321059
LR auc: 0.8816279194873802
KS: 0.7632558389747602


10. Credit score classification (XGB Model)

https://www.kaggle.com/datasets/parisrohan/credit-score-classification?select=train.csv

license: CC0: Public Domain

In [22]:
score = pandas.read_csv("/kaggle/input/datasets-for-federated-learning/wefe-default_data/score/train.csv")
score["y"] = score["Credit_Score"].apply(lambda x:1 if x == "Poor" else 0)
score = score.drop(columns = ["Credit_Score", "Customer_ID", "Name", "SSN", "Type_of_Loan", "Month", "Occupation"])
score

,ID,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,y
0,0x1602,23,19114.12,1824.843333,3,4,3,4,3,7,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,0
1,0x1603,23,19114.12,NaN,3,4,3,4,-1,NaN,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,0
2,0x1604,-500,19114.12,NaN,3,4,3,4,3,7,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,0
3,0x1605,23,19114.12,NaN,3,4,3,4,5,4,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,0
4,0x1606,23,19114.12,1824.843333,3,4,3,4,6,NaN,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,25,39628.99,3359.415833,4,6,7,2,23,7,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,1
99996,0x25fea,25,39628.99,3359.415833,4,6,7,2,18,7,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,1
99997,0x25feb,25,39628.99,3359.415833,4,6,5729,2,27,6,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,1
99998,0x25fec,25,39628.99,3359.415833,4,6,7,2,20,NaN,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,0


In [23]:
#preprocessing data
def preprocessing_score(score):
    
    def process_missing(x):
        if type(x) == str:
            if "_" in x:
                return x[:x.index("_")] + "0"
            else:
                return x
        else:
            return x

    def get_months(x):
        if type(x) == str:
            year = int(x[:x.index("Y") - 1])
            month = int(x[x.index("d") + 1:x.index("M") - 1])
            return year*12 + month
        else:
            return x
    
    def drop_(x):
        if type(x) == str:
            if "_" in x:
                return x[:-1]
            else:
                return x
        else:
            return x
    
    score["Num_of_Delayed_Payment"] = score["Num_of_Delayed_Payment"].apply(lambda x: process_missing(x))
    score["Num_of_Delayed_Payment"] = score["Num_of_Delayed_Payment"].apply(lambda x:int(x) if type(x) == str else x)
    score["Num_of_Delayed_Payment"] = score["Num_of_Delayed_Payment"].fillna(score["Num_of_Delayed_Payment"].median())    
    score["Credit_History_Age"] = score["Credit_History_Age"].apply(lambda x:get_months(x))
    fill = score[score["Amount_invested_monthly"] != "__10000__"]["Amount_invested_monthly"].apply(lambda x:float(x)).median()
    score["Amount_invested_monthly"] = score["Amount_invested_monthly"].apply(lambda x:fill if x == "__10000__" else x)
    score["Amount_invested_monthly"] = score["Amount_invested_monthly"].apply(lambda x:float(x))
    fill = score[score["Monthly_Balance"] != "__-333333333333333333333333333__"]["Monthly_Balance"].apply(lambda x:float(x)).mean()
    score["Monthly_Balance"] = score["Monthly_Balance"].apply(lambda x: fill if x == "__-333333333333333333333333333__" else x)
    score["Monthly_Balance"] = score["Monthly_Balance"].apply(lambda x:float(x))
    score["Annual_Income"] = score["Annual_Income"].apply(lambda x:drop_(x))
    score["Annual_Income"] = score["Annual_Income"].apply(lambda x:float(x))
    score["Age"] = score["Age"].apply(lambda x:drop_(x))
    score["Age"] = score["Age"].apply(lambda x:int(x))
    score["Age"] = score["Age"].apply(lambda x:score["Age"].median() if x == -500 else x)
    score["Num_of_Loan"] = score["Num_of_Loan"].apply(lambda x:drop_(x))
    score["Num_of_Loan"] = score["Num_of_Loan"].apply(lambda x:int(x))
    score["Outstanding_Debt"] = score["Outstanding_Debt"].apply(lambda x:drop_(x))
    score["Outstanding_Debt"] = score["Outstanding_Debt"].apply(lambda x:float(x))
    fill = score[score["Changed_Credit_Limit"] != "_"]["Changed_Credit_Limit"].apply(lambda x:float(x)).mean()
    score["Changed_Credit_Limit"] = score["Changed_Credit_Limit"].apply(lambda x:fill if x == "_" else x)
    score["Changed_Credit_Limit"] = score["Changed_Credit_Limit"].apply(lambda x:float(x))
    
    return score

score = preprocessing_score(score)

In [24]:
#filling missing value
score["Monthly_Inhand_Salary"] = score["Monthly_Inhand_Salary"].fillna(score["Monthly_Inhand_Salary"].median())
score["Num_of_Delayed_Payment"] = score["Num_of_Delayed_Payment"].fillna(score["Num_of_Delayed_Payment"].median())
score["Num_Credit_Inquiries"] = score["Num_Credit_Inquiries"].fillna(score["Num_Credit_Inquiries"].median())
score["Credit_History_Age"] = score["Credit_History_Age"].fillna(score["Credit_History_Age"].median())
score["Num_Credit_Inquiries"] = score["Num_Credit_Inquiries"].fillna(score["Num_Credit_Inquiries"].median())
score["Amount_invested_monthly"] = score["Amount_invested_monthly"].fillna(score["Amount_invested_monthly"].median())
score["Monthly_Balance"] = score["Monthly_Balance"].fillna(score["Monthly_Balance"].mean())

#woe coding
to_woe = [i for i in score.columns[1:-1] if len(score[i].unique()) <= 16]
processed = woe(score[to_woe + ["y"]])
score[to_woe + ["y"]] = processed

#standardization
score[[i for i in score.columns[1:-1] if len(score[i].unique()) > 16]] = score[[i for i in score.columns[1:-1] if len(score[i].unique()) > 16]].apply(lambda x:(x-x.mean())/x.std())

#data split
x = score[score.columns[1:-1]]
y = score[score.columns[-1]]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 2200)

#XGB
xgb_cls = xgboost.XGBClassifier()
xgb_clf = xgb_cls.fit(train_x, train_y)

#prediction, print evaluation results
xgb_predict_y = xgb_clf.predict(test_x)
fpr, tpr, threshold = sklearn.metrics.roc_curve(xgb_predict_y, test_y)
print("\nXGB accuracy: {}\nXGB auc: {}\nKS: {}".format(sum(xgb_predict_y == test_y)/len(xgb_predict_y), 
                                               sklearn.metrics.roc_auc_score(xgb_predict_y, test_y),
                                              max(tpr-fpr)))


XGB accuracy: 0.8469666666666666
XGB auc: 0.8186955359034327
KS: 0.6373910718068655
